Create a parcel-tra/hra/pda crosswalk that is consistent with Final Blueprint UrbanSim input. 

In [1]:
import pandas as pd
import os

In [2]:
# read parcel data with tra_id, hra_id and pda_id
parcel_file = 'C:\\Users\\{}\\Box\\Modeling and Surveys\\Urban Modeling\\Bay Area UrbanSim\\PBA50\\Current PBA50 Large General Input Data\\2021_02_25_parcels_geography.csv'.format(os.getenv('USERNAME'))
df_p = pd.read_csv(parcel_file,
                   usecols = ['PARCEL_ID', 'fbp_tra_id', 'fbp_sesit_id', 'pda_id_pba50_fb', 'juris'])
print('read {} rows of data from {}'.format(df_p.shape[0], parcel_file))

read 1956208 rows of data from C:\Users\ywang\Box\Modeling and Surveys\Urban Modeling\Bay Area UrbanSim\PBA50\Current PBA50 Large General Input Data\2021_02_25_parcels_geography.csv


In [3]:
# read parcel data with pda names
pda_name_file = 'M:\\Data\\GIS layers\\Blueprint Land Use Strategies\\ID_idx\\Final Blueprint\\2020_09_17_p10_pda_idx.csv'
df_pda = pd.read_csv(pda_name_file,
                     usecols = ['PARCEL_ID', 'pda_name'])
print('read {} rows of data from {}'.format(df_pda.shape[0], pda_name_file))

read 1956208 rows of data from M:\Data\GIS layers\Blueprint Land Use Strategies\ID_idx\Final Blueprint\2020_09_17_p10_pda_idx.csv


In [4]:
# merge
df = df_p.merge(df_pda, on='PARCEL_ID', how='inner')

In [5]:
# benchmark
print(df.fbp_tra_id.value_counts())
print(df.fbp_sesit_id.value_counts())

tra3     512800
tra2c    127874
tra2a     47445
tra1      40522
tra2b     37889
Name: fbp_tra_id, dtype: int64
DIS       580875
HRA       435620
HRADIS    407566
Name: fbp_sesit_id, dtype: int64


In [6]:
# configure the tra_id field
df.loc[df.fbp_tra_id.isnull(), 'fbp_tra_id'] = 'non-tra'

# create an tra_cat_id field
df['fbp_tra_cat_id'] = df['fbp_tra_id']
df.loc[df.fbp_tra_id.isin(['tra2a', 'tra2b', 'tra2c']), 'fbp_tra_cat_id'] = 'tra2'

# create an hra_id field
df['fbp_hra_id'] = 'non-HRA'
df.loc[(df.fbp_sesit_id == 'HRADIS') | (df.fbp_sesit_id == 'HRA'), 'fbp_hra_id'] = 'HRA'

In [7]:
# verify
print(df.fbp_tra_id.value_counts())
print(df.fbp_tra_cat_id.value_counts())
print(df.fbp_hra_id.value_counts())

non-tra    1189678
tra3        512800
tra2c       127874
tra2a        47445
tra1         40522
tra2b        37889
Name: fbp_tra_id, dtype: int64
non-tra    1189678
tra3        512800
tra2        213208
tra1         40522
Name: fbp_tra_cat_id, dtype: int64
non-HRA    1113022
HRA         843186
Name: fbp_hra_id, dtype: int64


In [8]:
# remove extra field
df.drop('fbp_sesit_id', axis=1, inplace=True)

In [9]:
# add synthetic parcels with job/hh

syn_parcels = {'PARCEL_ID':      [2054504, 2054505, 2054506, 572927],
               'fbp_tra_id':     ['non-tra', 'non-tra', 'non-tra', 'non-tra'],
               'fbp_tra_cat_id': ['non-tra', 'non-tra', 'non-tra', 'non-tra'],
               'fbp_hra_id':     ['non-HRA', 'non-HRA', 'non-HRA', 'non-HRA'],
               'juris':          ['unincorporated_marin', 'unincorporated_santa_clara', 'unincorporated_marin', 'unincorporated_contra_costa']}
syn_df = pd.DataFrame(data=syn_parcels)
display(syn_df)

df = pd.concat([df, syn_df])

,PARCEL_ID,fbp_tra_id,fbp_tra_cat_id,fbp_hra_id,juris
0,2054504,non-tra,non-tra,non-HRA,unincorporated_marin
1,2054505,non-tra,non-tra,non-HRA,unincorporated_santa_clara
2,2054506,non-tra,non-tra,non-HRA,unincorporated_marin
3,572927,non-tra,non-tra,non-HRA,unincorporated_contra_costa


In [10]:
# save to M
print(df.dtypes)
output_file = 'M:\\Data\\GIS layers\\Blueprint Land Use Strategies\\ID_idx\\Final Blueprint\\parcel_tra_hra_pda_fbp_20210816.csv'
print('export {} rows of data to {}'.format(df.shape[0], output_file))
df[['PARCEL_ID', 'juris', 'fbp_tra_id', 'fbp_tra_cat_id', 'fbp_hra_id', 'pda_id_pba50_fb', 'pda_name']].to_csv(output_file, index=False)

PARCEL_ID           int64
juris              object
pda_id_pba50_fb    object
fbp_tra_id         object
pda_name           object
fbp_tra_cat_id     object
fbp_hra_id         object
dtype: object
export 1956212 rows of data to M:\Data\GIS layers\Blueprint Land Use Strategies\ID_idx\Final Blueprint\parcel_tra_hra_pda_fbp_20210816.csv
